# Tribble Status

shows current status of jubilant tribble application

## Open Sessions

In [ ]:
import sqlite3
import os
import time
from IPython.display import clear_output

tribble_db_fqn = f"{os.getenv('JUBILANT_TRIBBLE_DATABASE')}"

con = sqlite3.connect(tribble_db_fqn)

cur = con.cursor()

while True:
    clear_output(wait=True)
    res = cur.execute("SELECT symbol from worktime w WHERE w.timeslot_finish == FALSE;")
    results_list = res.fetchall()
    print('Folowing Sessions are open:')
    for result in results_list:
        print(f"- {result[0]}")
    time.sleep(1)
